In [ ]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

[nltk_data] Downloading package stopwords to /Users/saum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/saum/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/saum/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


DATA PREPROCESSING

In [ ]:
df_train = pd.read_csv('review_train.csv')
df_test = pd.read_csv('review_test.csv')

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
def preprocess_text(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.lower() not in stop_words and word.lower() not in string.punctuation]
    words = [stemmer.stem(word) for word in words]
    preprocessed_text = ' '.join(words)
    return preprocessed_text

def convert_rating_to_numeric(rating_str):
    return int(rating_str.split()[0])
df_train['Rating'] = df_train['Rating'].apply(convert_rating_to_numeric).copy()

In [ ]:
df1 = df_train
df2 = df_test

In [ ]:
X = df1['Review']
y = df1['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_preprocessed = [preprocess_text(review) for review in X_train]
X_test_preprocessed = [preprocess_text(review) for review in X_test]

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train_preprocessed)
X_test_tfidf = vectorizer.transform(X_test_preprocessed)

model = AdaBoostClassifier(n_estimators=300, learning_rate=0.4)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.62      0.36      0.46       415
           2       0.42      0.50      0.45       454
           3       0.39      0.44      0.41       396
           4       0.38      0.52      0.44       370
           5       0.67      0.48      0.56       380

    accuracy                           0.46      2015
   macro avg       0.49      0.46      0.46      2015
weighted avg       0.49      0.46      0.46      2015



In [ ]:
base_estimator = DecisionTreeClassifier(max_depth=3)
model = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=300,
    learning_rate=0.01,
    random_state=42
)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           1       0.59      0.30      0.40       415
           2       0.35      0.51      0.41       454
           3       0.35      0.27      0.30       396
           4       0.32      0.46      0.38       370
           5       0.58      0.44      0.50       380

    accuracy                           0.40      2015
   macro avg       0.44      0.40      0.40      2015
weighted avg       0.44      0.40      0.40      2015



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.3],
    'base_estimator__max_depth': [3, 5, 7]
}

base_estimator = DecisionTreeClassifier()
ada = AdaBoostClassifier(base_estimator=base_estimator)
grid_search = GridSearchCV(ada, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_tfidf, y_train)
best_model = grid_search.best_estimator_
best_model.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = best_model.predict(X_test_tfidf)

# Print classification report
print("Best Parameters:", best_params)
print("Classification Report:")
print(classification_report(y_test, y_pred))


/Users/saum/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/saum/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/saum/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/saum/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/saum/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.w

NameError: name 'best_params' is not defined

In [ ]:
best_params = grid_search.best_params_

RUNNING THE ADABOOST MODEL, SENTIMENT ANALYSIS

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer
from scipy.sparse import hstack
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

sentiment_analyzer = SentimentIntensityAnalyzer()
def extract_sentiment_features(texts):
    sentiment_features = []
    for text in texts:
        sentiment_score = sentiment_analyzer.polarity_scores(text)
        sentiment_features.append([sentiment_score['neg'], sentiment_score['neu'], sentiment_score['pos'], sentiment_score['compound']])
    return sentiment_features

X_train_sentiment = extract_sentiment_features(X_train)
X_test_sentiment = extract_sentiment_features(X_test)

X_train_combined = hstack([X_train_tfidf, X_train_sentiment])
X_test_combined = hstack([X_test_tfidf, X_test_sentiment])

model = AdaBoostClassifier(
    n_estimators=500,
    learning_rate=0.05,

    random_state=42
)

model.fit(X_train_combined, y_train)

y_pred = model.predict(X_test_combined)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           1       0.61      0.46      0.52       415
           2       0.39      0.47      0.42       454
           3       0.38      0.33      0.35       396
           4       0.40      0.54      0.46       370
           5       0.65      0.53      0.58       380

    accuracy                           0.46      2015
   macro avg       0.49      0.46      0.47      2015
weighted avg       0.48      0.46      0.47      2015



HYPERPARAMETER TUNING

In [ ]:
print("Best Parameters:", best_params)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Best Parameters: {'base_estimator__max_depth': 3, 'learning_rate': 0.3, 'n_estimators': 50}
Classification Report:
              precision    recall  f1-score   support

           1       0.58      0.33      0.42       415
           2       0.39      0.48      0.43       454
           3       0.35      0.39      0.37       396
           4       0.35      0.51      0.42       370
           5       0.64      0.42      0.50       380

    accuracy                           0.42      2015
   macro avg       0.46      0.42      0.43      2015
weighted avg       0.46      0.42      0.43      2015

